In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import gc
from collections import defaultdict
from tqdm.notebook import tqdm
import lightgbm as lgb

# Download kaggle data

In [ ]:
!kaggle datasets download -f cv1_train.pickle its7171/riiid-cross-validation-files
!kaggle datasets download -f cv1_valid.pickle its7171/riiid-cross-validation-files
!kaggle competitions download -f questions.csv riiid-test-answer-prediction
!kaggle competitions download -f lectures.csv riiid-test-answer-prediction

100%|█████████████████████████████████████▉| 1.64G/1.64G [00:20<00:00, 68.2MB/s]
100%|██████████████████████████████████████| 1.64G/1.64G [00:20<00:00, 87.6MB/s]
 82%|███████████████████████████████       | 33.0M/40.4M [00:01<00:00, 24.2MB/s]
100%|██████████████████████████████████████| 40.4M/40.4M [00:01<00:00, 28.0MB/s]
  0%|                                                | 0.00/289k [00:00<?, ?B/s]
100%|████████████████████████████████████████| 289k/289k [00:00<00:00, 7.21MB/s]
  0%|                                               | 0.00/9.48k [00:00<?, ?B/s]
100%|██████████████████████████████████████| 9.48k/9.48k [00:00<00:00, 10.0MB/s]


In [ ]:
!mkdir train
!unzip \*.zip -d train && rm *.zip

Archive:  cv1_train.pickle.zip
  inflating: train/cv1_train.pickle  

Archive:  cv1_valid.pickle.zip
  inflating: train/cv1_valid.pickle  

2 archives were successfully processed.


In [ ]:
%%time
columns = ['timestamp', 'user_id', 'content_id', 'content_type_id', 'task_container_id', 'answered_correctly', 'prior_question_elapsed_time', 'prior_question_had_explanation']
train_df = pd.read_pickle('train/cv1_train.pickle')[columns]
valid_df = pd.read_pickle('train/cv1_valid.pickle')[columns]

print('train_df size: {}'.format(train_df.shape))
print('valid_df size: {}'.format(valid_df.shape))

train_df size: (98730332, 8)
valid_df size: (2500000, 8)
CPU times: user 2.05 s, sys: 6.41 s, total: 8.46 s
Wall time: 8.46 s


In [ ]:
prior_question_elapsed_time_mean = train_df['prior_question_elapsed_time'].dropna().values.mean()
prior_question_elapsed_time_mean

25439.41